# GCP Pub/Sub QuickStart 
https://cloud.google.com/pubsub/docs/quickstart-client-libraries

## Setup the Notebook Environment

In [1]:
!gcloud projects list

PROJECT_ID                NAME                PROJECT_NUMBER
bq-sandbox-275522         BQ Sandbox          825850861083
homelab-266121            HomeLab             18021674906
minecraftserver-237915    MinecraftServer     228956756515
quickstart-1588989022480  Quickstart          34250947427
sandbox-project-188621    Sandbox project     860402107822
storage-project-01        storage-project-01  549499142500
trafficapp-214315         TrafficApp          398258726631
virtualtoolbox            VirtualToolbox      1048728763860
Exception ignored in: <function _after_fork at 0x1050f7310>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 1486, in _after_fork
    thread._reset_internal_locks(True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 829, in _reset_internal_locks
    self._tstate_lock._at_fork_reinit()
AttributeError: 'NoneType' object has no attribute '_at_for

In [2]:
# change these to try this notebook out
PROJECT = 'sandbox-project-188621'
import os
os.environ['PROJECT'] = PROJECT

dir_root = './'

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config list project

[core]
project = sandbox-project-188621


Updated property [core/project].
Exception ignored in: <function _after_fork at 0x10d9f9310>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 1486, in _after_fork
    thread._reset_internal_locks(True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 829, in _reset_internal_locks
    self._tstate_lock._at_fork_reinit()
AttributeError: 'NoneType' object has no attribute '_at_fork_reinit'

Your active configuration is: [default]
Exception ignored in: <function _after_fork at 0x109943310>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 1486, in _after_fork
    thread._reset_internal_locks(True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 829, in _reset_internal_locks
    self._tstate_lock._at_fork_reinit()
AttributeError: 'NoneType' object has n

In [4]:
API_KEY = "sandbox-project-188621-3ac36bf64693.json"

## Creating a topic and a subscription

In [5]:
!gcloud pubsub topics create my-topic

ERROR: Failed to create topic [projects/sandbox-project-188621/topics/my-topic]: Resource already exists in the project (resource=my-topic).
ERROR: (gcloud.pubsub.topics.create) Failed to create the following: [my-topic].
Exception ignored in: <function _after_fork at 0x10c333310>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 1486, in _after_fork
    thread._reset_internal_locks(True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 829, in _reset_internal_locks
    self._tstate_lock._at_fork_reinit()
AttributeError: 'NoneType' object has no attribute '_at_fork_reinit'


In [6]:
!gcloud pubsub subscriptions create my-sub --topic my-topic

ERROR: Failed to create subscription [projects/sandbox-project-188621/subscriptions/my-sub]: Resource already exists in the project (resource=my-sub).
ERROR: (gcloud.pubsub.subscriptions.create) Failed to create the following: [my-sub].
Exception ignored in: <function _after_fork at 0x1059c1310>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 1486, in _after_fork
    thread._reset_internal_locks(True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 829, in _reset_internal_locks
    self._tstate_lock._at_fork_reinit()
AttributeError: 'NoneType' object has no attribute '_at_fork_reinit'


## Publishing messages

In [22]:
from google.cloud import pubsub_v1

# TODO(developer)
project_id = PROJECT
topic_id = "my-topic"

publisher = pubsub_v1.PublisherClient.from_service_account_json(API_KEY)
# The `topic_path` method creates a fully qualified identifier
# in the form `projects/{project_id}/topics/{topic_id}`
topic_path = publisher.topic_path(project_id, topic_id)

for n in range(1, 10):
    data = "Message number {}".format(n)
    # Data must be a bytestring
    data = data.encode("utf-8")
    # When you publish a message, the client returns a future.
    future = publisher.publish(topic_path, data)
    print(future.result())

print(f"Published messages to {topic_path}.")

1855326548344126
1855322833530451
1855325102684012
1855304169376649
1855331719643572
1855302720443937
1855314493513057
1855318161528757
1855321855457245
Published messages to projects/sandbox-project-188621/topics/my-topic.


## Receiving messages

In [51]:
from concurrent.futures import TimeoutError
from google.cloud import pubsub_v1

# TODO(developer)
# project_id = "your-project-id"
project_id = PROJECT
subscription_id = "my-sub"
# Number of seconds the subscriber should listen for messages
timeout = 5.0

subscriber = pubsub_v1.SubscriberClient.from_service_account_json(API_KEY)
# The `subscription_path` method creates a fully qualified identifier
# in the form `projects/{project_id}/subscriptions/{subscription_id}`
subscription_path = subscriber.subscription_path(project_id, subscription_id)

results = []

def callback(message):
    results.append(message)
    print(f"Received {message}.")
    message.ack()

streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

# Wrap subscriber in a 'with' block to automatically call close() when done.
with subscriber:
    try:
        # When `timeout` is not set, result() will block indefinitely,
        # unless an exception is encountered first.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()

Listening for messages on projects/sandbox-project-188621/subscriptions/my-sub..

Received Message {
  data: b'{"a":{"0":1,"1":1},"b":{"0":2,"1":2}}'
  ordering_key: ''
  attributes: {}
}.
Received Message {
  data: b'{"a":{"0":1,"1":1},"b":{"0":2,"1":2}}'
  ordering_key: ''
  attributes: {}
}.


In [39]:
results[0].data

b'Message number 2'

## Passing DF/JSON Data

In [41]:
import pandas as pd
df = pd.DataFrame(columns=['a', 'b'], data=[[1,2], [1,2]])

In [43]:
df.to_json()

'{"a":{"0":1,"1":1},"b":{"0":2,"1":2}}'

In [52]:
data = df.to_json()
# Data must be a bytestring
data = data.encode("utf-8")
# When you publish a message, the client returns a future.
future = publisher.publish(topic_path, data)
print(future.result())

1855332609439853


#### *Use subscriber block to pull the latest messages*

In [67]:
results

[Message {
   data: b'{"a":{"0":1,"1":1},"b":{"0":2,"1":2}}'
   ordering_key: ''
   attributes: {}
 },
 Message {
   data: b'{"a":{"0":1,"1":1},"b":{"0":2,"1":2}}'
   ordering_key: ''
   attributes: {}
 }]

In [65]:
pd.read_json(results[0].data)

,a,b
0,1,2
1,1,2
